<div align="center"; span style="color:#336699"><b><h2>pyForTraCC - Track Infrared (Real Time Data) </h2></b></div>
<hr style="border:2px solid #0077b9;">
<br/>
<div style="text-align: center;font-size: 90%;">
   <sup><a href="https://www.linkedin.com/in/helvecio-leal/"> Helvécio B. Leal Neto, <i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup><t>&nbsp;</t> 
    <sup><a href="https://www.linkedin.com/in/alan-calheiros-64a252160/">Alan J. P. Calheiros<i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
   <br/><br/>
    National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:helvecio.neto@inpe.br">helvecio.neto@inpe.br</a>, <a href="mailto:alan.calheiros@inpe.br">alan.calheiros@inpe.br</a>
    <br/><br/>
    Last Update: Apr 4, 2025
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Abstract.</b> This Jupyter Notebook shows how to use pyfortracc to track latest images from the GOES-19 satellite.<br>
The algorithm uses the brightness temperature data from the ABI sensor to identify and track precipitating systems over the Brazil.<br>
The output data is a tracking table containing the system's lifecyle and characteristics.
</div>    
<br/>
<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>In this example, we will use fortracc to compute track of precipitating systems over the globe and explore the output data after the algorithm workflow.
</b>
    <div style="margin-left: 10px; margin-right: 10px; margin-top:10px">
      <p> Leal Neto, H.B.; Calheiros, A.J.P.;  fortracc Algorithm. São José dos Campos, INPE, 2024. <a href="https://github.com/fortracc-project/" target="_blank"> Online </a>. </p>
    </div>
</div>

### Schedule

### Schedule
 [1. Installation](#install)<br>
 [2. Input Data](#input)<br>
 [3. Read Function](#data)<br>
 [4. Parameters (Name_list)](#namelist)<br>
 [5. Tracking Routine](#track)<br>
 [6. Tracking Visualization](#visualization)<br>

<a id='install'></a>
#### 1. Installation

The installation of the pyfortracc package can be done using the pip command. 

In [ ]:
# Install latest version of pyfortracc
!python -m pip install -q -U pyfortracc

<a id='data'></a>
#### 2. Data Input (Download Files)

We use 'goesgcp' package to download the latest GOES-19 data from Google Cloud Platform (GCP).

For install the package, use the command below:

In [ ]:
# Install goesgcp from pypi
!python -m pip install -q -U goesgcp

After the installation of goesgcp, we can download the latest 10 images from the GOES-19 satellite and reproject the data to a regular grid with 0.045 degree resolution, approximately 5 km. And crop the data to the Brazil region.

In [ ]:
# Remove any previous input/
!rm -rf input/

# Download recent 10 GOES-19 images
!goesgcp --recent 10 --output "input/" --resolution 0.045 --lat_min -35 --lat_max 5 --lon_min -80 --lon_max -30

<a id='namelist'></a>
#### 3. Read Function

The `read_function` function is used to read the data from the input files. The function returns the data a numpy array of the brightness temperature data.

In [1]:
# The function below reads the data from the downloaded files
import xarray as xr
import glob

def read_function(path):
	ds=xr.open_dataset(path)
	return ds['CMI'].data[::-1]

To set the boundaries of the data, we use read_function to read the first image and get the latitude and longitude values.

In [2]:
# Set the lon_min, lon_max, lat_min and lat_max of domain
files = glob.glob('input/*.nc')
ds = xr.open_dataset(files[0])

# Get the lon_min, lon_max, lat_min and lat_max of the domain
lon_min = float(ds['lon'].min().values)
lon_max = float(ds['lon'].max().values)
lat_min = float(ds['lat'].min().values)
lat_max = float(ds['lat'].max().values)

<a id='namelist'></a>
#### 4. Parameters: name_list

The `name_list` is a dictionary with the parameters to be used in the tracking algorithm. 
For this example, we will use the parameters associated with mesoscale convective systems (MCS).
We track the MCS using the brightness temperature data from the ABI sensor. And set the threshold values to the warmest to coldest temperatures by brightness temperature values of 235 K to 200 K. And set the minimum area of the system to 400 pixels (approximately 10.000 km²) and the next minimum area to 100 pixels (approximately 2.500 km²).

In [ ]:
# Set name_list dictionary
name_list = {}

# Mandatory parameters
name_list['input_path'] = 'input/' # Set the input path
name_list['output_path'] = 'output/' # Set the output path
name_list['thresholds'] = [235, 200] # Set the thresholds for tracking
name_list['min_cluster_size'] = [400, 100] # Set the minimum size each individual cluster
name_list['operator'] = '<=' # Set the operator segmentation based on the thresholds
name_list['timestamp_pattern'] = 'OR_ABI-L2-CMIPF-M6C13_G19_s%Y%j%H%M' # Set the timestamp pattern of filenames
name_list['pattern_position'] = [0, 38] # Set the pattern position, this is the position of the timestamp in the filename
name_list['delta_time'] = 10 # Set the delta time between images

# Spatial parameters
name_list['lon_min'] = lon_min # Set the lon_min
name_list['lon_max'] = lon_max # Set the lon_max
name_list['lat_min'] = lat_min # Set the lat_min
name_list['lat_max'] = lat_max # Set the lat_max

# Vector method parameters (showed in article: https://doi.org/10.3390/rs14215408)
name_list['spl_correction'] = True # It is used to perform the correction at Splitting events
name_list['mrg_correction'] = True # It is used to perform the correction at Merging events
name_list['inc_correction'] = True # It is used to perform the correction using Inner Core vectors
name_list['opt_correction'] = True # It is used to perform the correction using the Optical Flow method
name_list['elp_correction'] = True # It is used to perform the correction using the Ellipse method
name_list['validation'] = True # It is used to perform the validation of the correction methods

<a id='track'></a>
#### 5. Track Routine

The `track` function is a main funtion to track the MCS from pyfortracc package. 

In [18]:
# Import track function from pyfortracc
from pyfortracc import track

In [ ]:
# Track the clusters 
track(name_list, read_function)

<a id='visualize'></a>
#### 6. Track Output

The outputs of the `track` function is the `tracking_table`, which contains the tracking information of the MSCs.

The `tracking_table` is a DataFrame containing the tracking information of the convective systems.<br>

To load the tracking table, we use the `duckdb` witch is library to load the parquet files and query the data.

In [ ]:
# Import duckdb
import duckdb

# Connect to the database
con = duckdb.connect(database=':memory:', read_only=False)

# Open tracking table
tracking_table = con.execute(f"""SELECT * 
                             FROM parquet_scan('output/track/trackingtable/*.parquet',
                             union_by_name=True)
                             """).fetch_df()

Other way to load the tracking table is using the `plot_animation` function from the `pyfortracc` module.

The `plot_animation` receives the data and the track as input and plots the data and the track on the same map.<br>
We need to set the dimensions of the plot, the projection, and the extent of the plot.

In [21]:
# Import numpy for np.greater_equal and plot_animation from pyfortracc
import numpy as np
from pyfortracc import plot_animation

In [ ]:
# Visualize as animation.
plot_animation(read_function=read_function, # Read function
                          figsize=(8,8), # Figure size
                          name_list=name_list, # Name list dictionary
                          start_timestamp = str(tracking_table['timestamp'].min()), # Start timestamp
                          end_timestamp= str(tracking_table['timestamp'].max()), # End timestamp
                          info_col_name=False, # Info column name
                          cbar_title='Temperature(k)', # Colorbar title
                          trajectory=True, # Plot the trajectory
                          smooth_trajectory=True, # Smooth the trajectory
                          cmap='turbo', # Colormap
                          min_val=200, # Min value
                          max_val=235, # Max value
                          nan_value=235, # NaN value
                          nan_operation=np.greater_equal, # NaN operation
                          bound_color='blue', # Bound color
                          info_cols=['uid'], # Info columns from tracking table
                          parallel=False
                          )